In [8]:
# Install the necessary package
!pip install pyswarms

# Import required libraries
import numpy as np
import pandas as pd
from pyswarms.single.global_best import GlobalBestPSO

# Define cost parameters for various work types
full_time_rates = np.array([30, 35, 40, 45, 50])                # Full-time hourly rates
overtime_rates = 1.5 * full_time_rates                            # Overtime rates (1.5 times the full-time rate)
part_time_rates = np.array([25, 27, 29, 31, 33])                  # Part-time hourly rates

# Combine all cost parameters into a single cost vector for the 15 decision variables
cost_vector = np.concatenate([full_time_rates, overtime_rates, part_time_rates])

def compute_total_labor_expenses(x):
    penalty_list = []  # List to store penalties for each solution
    for xi in x:
        xi = np.clip(xi, 0, None)  # Ensure all values are non-negative (no negative working hours)
        peak_hours = np.sum(xi[:5]) + np.sum(xi[5:10])  # Total hours for full-time and overtime
        non_peak_hours = np.sum(xi[10:15])  # Part-time hours

        penalty = 0
        if peak_hours < 4:
            penalty += (4 - peak_hours) * 1000  # Apply penalty if peak hours are less than 4
        if non_peak_hours < 2:
            penalty += (2 - non_peak_hours) * 1000  # Apply penalty if non-peak part-time hours are less than 2

        total_cost = np.sum(cost_vector * xi)  # Compute the total cost (labor cost without penalties)
        penalty_list.append(total_cost + penalty)  # Append the total cost with penalties
    return np.array(penalty_list)

# Define bounds for the optimization and PSO parameters
lower_bounds = np.zeros(15)  # Minimum working hours (0 hours for all types)
upper_bounds = np.ones(15) * 8  # Maximum working hours (8 hours per type)
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7}  # PSO algorithm settings

# Perform the Particle Swarm Optimization (PSO)
optimizer = GlobalBestPSO(n_particles=30, dimensions=15, options=options, bounds=(lower_bounds, upper_bounds))
best_cost, best_position = optimizer.optimize(compute_total_labor_expenses, iters=100)

# Define categories and hourly rates
categories = ['Full-time (x_i)', 'Overtime (x_io)', 'Part-time (x_jp)']
rates = np.concatenate([full_time_rates, overtime_rates, part_time_rates])

# Create a DataFrame to display results
df = pd.DataFrame({
    'Category': [categories[i // 5] for i in range(15)],  # Assign category labels
    'Hourly Rate': rates,
    'Hours Worked': best_position,
    'Cost': rates * best_position  # Calculate total cost based on hours worked
})

# Calculate the total peak and non-peak hours worked
total_peak_hours = np.sum(best_position[:10])  # Sum of all peak hours (full-time + overtime)
total_non_peak_hours = np.sum(best_position[10:15])  # Sum of non-peak (part-time)

# Print the results
print(f"Total Labor Cost: {best_cost:.2f} SAR")
print(f"Peak Hours Total: {total_peak_hours:.2f} hours (Required ≥ 4)")
print(f"Non-Peak Part-Time Hours: {total_non_peak_hours:.2f} hours (Required ≥ 2)")

# Display the DataFrame with detailed results
df


2025-04-20 21:50:40,803 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=1.01e+3
2025-04-20 21:50:41,801 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1012.5995384950405, best pos: [0.865251   1.500264   3.24312444 1.12039584 1.11865508 2.23098212
 3.38770341 1.0798282  0.49907699 0.80244544 1.73225307 2.53096098
 1.51155487 1.43961647 1.84976755]


Total Labor Cost: 1012.60 SAR
Peak Hours Total: 15.85 hours (Required ≥ 4)
Non-Peak Part-Time Hours: 9.06 hours (Required ≥ 2)


,Category,Hourly Rate,Hours Worked,Cost
0,Full-time (x_i),30.0,0.865251,25.957530
1,Full-time (x_i),35.0,1.500264,52.509240
2,Full-time (x_i),40.0,3.243124,129.724977
3,Full-time (x_i),45.0,1.120396,50.417813
4,Full-time (x_i),50.0,1.118655,55.932754
5,Overtime (x_io),45.0,2.230982,100.394195
6,Overtime (x_io),52.5,3.387703,177.854429
7,Overtime (x_io),60.0,1.079828,64.789692
8,Overtime (x_io),67.5,0.499077,33.687697
9,Overtime (x_io),75.0,0.802445,60.183408
